# Creació del dataset dels vols

Importem les llibreries necessaries

In [1]:
import pandas as pd
import numpy as np
import random
import datetime
import math
import time

Importem el dataset amb dades dels aeroports

In [2]:
data = pd.read_excel('world_capitals.xlsx', engine='openpyxl')

Analitzem les dades d'aquest dataset

In [3]:
print(data)

             CountryName   CapitalName  CapitalLatitude  CapitalLongitude  \
0              Argentina  Buenos Aires       -34.583333        -58.666667   
1              Australia      Canberra       -35.266667        149.133333   
2                Austria        Vienna        48.200000         16.366667   
3                Belgium      Brussels        50.833333          4.333333   
4                 Brazil      Brasilia       -15.783333        -47.916667   
5                 Canada        Ottawa        45.416667        -75.700000   
6                  Chile      Santiago       -33.450000        -70.666667   
7                  China       Beijing        39.916667        116.383333   
8               Colombia        Bogota         4.600000        -74.083333   
9                Croatia        Zagreb        45.800000         16.000000   
10                  Cuba        Havana        23.116667        -82.350000   
11        Czech Republic        Prague        50.083333         14.466667   

In [4]:
print(data.columns)

Index(['CountryName', 'CapitalName', 'CapitalLatitude', 'CapitalLongitude',
       'CapitalLatitude Radians', 'CapitalLongitude Radians', 'CountryCode',
       'ContinentName'],
      dtype='object')


-----------------------------

Realitzem totes les operacions necessaries per tal d'obtenir les dades del nou dataset

In [5]:
#SIZE OF THE DATASET
n=100000

flights = pd.DataFrame()

airspeed = 800
airline_list = ["Delta Air Lines","American Airlines Group","United Airlines Holdings","Lufthansa Group","Air France–KLM","Southwest Airlines","Turkish Airlines","China Eastern Airlines","IAG"]

t1=time.time()
for i in range(n):
    #Check that the distance between Origin and Destination is realistic
    distance = 0
    while not (200 < distance < 10000):
        origin = random.randint(0,len(data)-1)
        destination = random.randint(0,len(data)-1)
        
        #Compute distance
        temp1 = data["CapitalLatitude Radians"][destination] - data["CapitalLatitude Radians"][origin]
        temp2 = data["CapitalLongitude Radians"][destination] - data["CapitalLongitude Radians"][origin]
        temp3 = math.sin(temp1/2)**2 + math.cos(data["CapitalLatitude Radians"][origin])*math.cos(data["CapitalLatitude Radians"][destination])*math.sin(temp2/2)**2
        temp4 = 2*math.asin(math.sqrt(temp3))
        distance = round(temp4*6371,2)
    
    #Get number of stops with probablity
    if 0 < distance <= 2000:
        stops = 0
    elif 2000 < distance <= 5000:
        stops = np.random.choice(np.arange(0, 3), p=[0.6, 0.3, 0.1])
    else:
        stops = np.random.choice(np.arange(0, 3), p=[0.1, 0.5, 0.4])
    
    #Get class with probability
    flight_class = random.choices(["Economy","Business"], weights=(65, 35))[0]
    
    #Create random departure time
    dep_hour = str(random.randint(0,23))
    if len(dep_hour)==1:
        dep_hour = "0"+dep_hour
    dep_min = str(random.randint(0,5))+ str(random.choice(["0","5"]))
    dep_time = dep_hour+":"+dep_min    
    
    #Compute flight time
    f_time = round(distance/airspeed+0.5+2*stops,2) # +30 min of takeoff, +2 hours per stop
    hours = int(f_time)
    minutes = (f_time*60) % 60
    flight_time = "%02d:%02d" % (hours, minutes)    
    
    #Compute "Arr_time" based on "Dep_time" and flight time
    timeList = [dep_time, flight_time]
    mysum = datetime.timedelta()
    for t in timeList:
        (h, m) = t.split(':')
        mysum += datetime.timedelta(hours=int(h), minutes=int(m))

    if mysum.days>0:
        mysum -= datetime.timedelta(days=1)
    
    time = str(mysum)
    if len(str(mysum))==7:
        time = "0"+time
    
    #Create random date
    d1 = datetime.datetime.strptime('1/1/2023', '%d/%m/%Y')
    d2 = datetime.datetime.strptime('31/03/2023', '%d/%m/%Y')
    
    delta = d2 - d1
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    res_date = d1 + datetime.timedelta(seconds=random_second)
    date = str(res_date)[8:10]+"-"+str(res_date)[5:7]+"-"+str(res_date)[0:4]

    #Compute "Price" depending on "Class"
    if flight_class=="Economy":
        price = round(f_time*50*random.uniform(1,1.5),2)
    else:
        price = round(f_time*50*random.uniform(2.2,3.2),2)
    
    #Create outliers (lower price than normal every 100 flights)
    if i%100 == 0:
        price = round(price*random.uniform(0.5,0.8),2)
    
    #Add new data to flights dataset
    flights.at[i,"Flight ID"] = i
    flights.at[i,"Date"] = date
    flights.at[i,"Airline"] = random.choice(airline_list)
    flights.at[i,"Dep_time"] = dep_time
    flights.at[i,"Time_taken"] = flight_time
    flights.at[i,"Arr_time"] = time[:-3]
    flights.at[i,"Stops"] = stops
    flights.at[i,"Origin City"] = data["CapitalName"][origin]
    flights.at[i,"Origin Latitude Radians"] = data["CapitalLatitude Radians"][origin]
    flights.at[i,"Origin Longitude Radians"] = data["CapitalLongitude Radians"][origin]
    flights.at[i,"Destination City"] = data["CapitalName"][destination]
    flights.at[i,"Destination Latitude Radians"] = data["CapitalLatitude Radians"][destination]
    flights.at[i,"Destination Longitude Radians"] = data["CapitalLongitude Radians"][destination]
    flights.at[i,"Price"] = price
    flights.at[i,"Class"] = flight_class
    
    if i%10000==0:
        print("Iterations:",i)
        if i%50000==0:
            t2=time.time()
            print("Temps d'execució de les últimes 50k iteracions:",t2-t1)
            t1=time.time()

print(flights)

Iterations: 0
Iterations: 10000
Iterations: 20000
Iterations: 30000
Iterations: 40000
Iterations: 50000
Iterations: 60000
Iterations: 70000
Iterations: 80000
Iterations: 90000
Iterations: 100000
Iterations: 110000
Iterations: 120000
Iterations: 130000
Iterations: 140000
Iterations: 150000
Iterations: 160000
Iterations: 170000
Iterations: 180000
Iterations: 190000
Iterations: 200000
Iterations: 210000
Iterations: 220000
Iterations: 230000
Iterations: 240000
Iterations: 250000
Iterations: 260000
Iterations: 270000
Iterations: 280000
Iterations: 290000
Iterations: 300000
Iterations: 310000
Iterations: 320000
Iterations: 330000
Iterations: 340000
Iterations: 350000
Iterations: 360000
Iterations: 370000
Iterations: 380000
Iterations: 390000
Iterations: 400000
Iterations: 410000
Iterations: 420000
Iterations: 430000
Iterations: 440000
Iterations: 450000
Iterations: 460000
Iterations: 470000
Iterations: 480000
Iterations: 490000
        Flight ID        Date                   Airline Dep_time

Columnes del dataset final

In [6]:
print(flights.columns)

Index(['Flight ID', 'Date', 'Airline', 'Dep_time', 'Time_taken', 'Arr_time',
       'Stops', 'Origin City', 'Origin Latitude Radians',
       'Origin Longitude Radians', 'Destination City',
       'Destination Latitude Radians', 'Destination Longitude Radians',
       'Price', 'Class'],
      dtype='object')


Exporto el nou dataset a Excel

In [7]:
flights.to_excel("dataset_100k.xlsx")